In [1]:
import pandas as pd
import re
import gensim
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from gensim.models.ldamodel import LdaModel,CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

C:\Users\samarth152721\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df2 = pd.read_csv('C:/Users/samarth152721/Downloads/complaints (1).csv')
df = df2[['Product','Consumer complaint narrative']]
df.dropna(inplace = True)
df.reset_index(drop = True,inplace = True)
df.head()

,Product,Consumer complaint narrative
0,Checking or savings account,On XX/XX/XXXX XXXX XXXX XXXX XXXX was set up b...
1,Checking or savings account,My parents both passed away ( Father- XX/XX/XX...
2,Checking or savings account,"Hello, I am submitting a complaint regarding a..."
3,Checking or savings account,On XX/XX/18 i placed an order with the company...
4,Checking or savings account,"On XX/XX/2019 at approximately XXXX XXXX EST, ..."


In [3]:
def renaming(text):
    if text =='Prepaid card':
        return 'Credit card or prepaid card'
    elif text == 'Payday loan':
        return 'Loans'
    elif text == 'Credit reporting':
        return 'Credit reporting, credit repair services, or other personal consumer reports'
    elif text == 'Money transfers':
        return 'Money transfer, virtual currency, or money service'
    elif text == 'Credit card':
        return 'Credit card or prepaid card'
    elif text == 'Consumer Loan':
        return 'Loans'
    elif text == 'Vehicle loan or lease':
        return 'Loans'
    elif text == 'Payday loan, title loan, or personal loan':
        return 'Loans'
    elif text == 'Student loan':
        return 'Loans'
    elif text == 'Virtual currency':
        return 'Money transfer, virtual currency, or money service'
    else:
        return text

In [4]:
df['Product'] = df['Product'].progress_apply(renaming)

100%|████████████████████████████████████████████████████████████████████████████████████████| 236/236 [00:00<?, ?it/s]


In [5]:
df['Product'].value_counts()

Checking or savings account    236
Name: Product, dtype: int64

In [6]:
df2 = df

In [7]:
df2.head()

,Product,Consumer complaint narrative
0,Checking or savings account,On XX/XX/XXXX XXXX XXXX XXXX XXXX was set up b...
1,Checking or savings account,My parents both passed away ( Father- XX/XX/XX...
2,Checking or savings account,"Hello, I am submitting a complaint regarding a..."
3,Checking or savings account,On XX/XX/18 i placed an order with the company...
4,Checking or savings account,"On XX/XX/2019 at approximately XXXX XXXX EST, ..."


In [8]:
stop_words = list(set(stopwords.words('english')))+list(punctuation)
lem = WordNetLemmatizer()

In [9]:
def cleaning(text):
    text = re.sub('XXXX',' UNKNOWN ',text)
    text = re.sub('XX/XX/','',text)
    text = re.sub('UNKNOWN   UNKNOWN','UNKNOWN',text)
    text = re.sub('UNKNOWN','',text)
    text = re.sub('\n',' ',text)
    text = re.sub('  ',' ',text)
    text = text.lower()
    words = word_tokenize(text)
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if len(w)>2]
    words = [lem.lemmatize(w) for w in words]
    return words

In [10]:
df2['clean_text'] = df2['Consumer complaint narrative'].progress_apply(cleaning)

100%|████████████████████████████████████████████████████████████████████████████████| 236/236 [00:02<00:00, 98.95it/s]


In [11]:
clean_doc = list(df2['clean_text'].values)

In [12]:
dictionary = Dictionary(clean_doc)

In [13]:
corpus = [dictionary.doc2bow(doc) for doc in clean_doc]

In [14]:
ldamodel = LdaModel(corpus=corpus,id2word=dictionary,num_topics=5,random_state=42,update_every=1,passes=100, chunksize=100)

In [15]:
print(ldamodel.print_topics())

[(0, '0.048*"pnc" + 0.041*"account" + 0.032*"bank" + 0.018*"check" + 0.011*"would" + 0.010*"received" + 0.010*"email" + 0.010*"fraud" + 0.009*"told" + 0.008*"said"'), (1, '0.016*"mom" + 0.010*"access" + 0.008*"cousin" + 0.007*"company" + 0.007*"log" + 0.006*"passed" + 0.006*"stock" + 0.006*"care" + 0.006*"court" + 0.005*"changed"'), (2, '0.039*"account" + 0.032*"pnc" + 0.015*"niece" + 0.014*"service" + 0.011*"customer" + 0.010*"bank" + 0.009*"would" + 0.008*"issue" + 0.007*"interest" + 0.007*"rate"'), (3, '0.059*"fee" + 0.040*"account" + 0.026*"overdraft" + 0.024*"charged" + 0.019*"charge" + 0.013*"balance" + 0.013*"pnc" + 0.012*"transaction" + 0.011*"bank" + 0.010*"time"'), (4, '0.035*"account" + 0.024*"pnc" + 0.022*"bank" + 0.016*"money" + 0.012*"told" + 0.012*"would" + 0.011*"call" + 0.011*"fund" + 0.011*"called" + 0.009*"time"')]


In [16]:
ldamodel.show_topics(formatted=True)

[(0,
  '0.048*"pnc" + 0.041*"account" + 0.032*"bank" + 0.018*"check" + 0.011*"would" + 0.010*"received" + 0.010*"email" + 0.010*"fraud" + 0.009*"told" + 0.008*"said"'),
 (1,
  '0.016*"mom" + 0.010*"access" + 0.008*"cousin" + 0.007*"company" + 0.007*"log" + 0.006*"passed" + 0.006*"stock" + 0.006*"care" + 0.006*"court" + 0.005*"changed"'),
 (2,
  '0.039*"account" + 0.032*"pnc" + 0.015*"niece" + 0.014*"service" + 0.011*"customer" + 0.010*"bank" + 0.009*"would" + 0.008*"issue" + 0.007*"interest" + 0.007*"rate"'),
 (3,
  '0.059*"fee" + 0.040*"account" + 0.026*"overdraft" + 0.024*"charged" + 0.019*"charge" + 0.013*"balance" + 0.013*"pnc" + 0.012*"transaction" + 0.011*"bank" + 0.010*"time"'),
 (4,
  '0.035*"account" + 0.024*"pnc" + 0.022*"bank" + 0.016*"money" + 0.012*"told" + 0.012*"would" + 0.011*"call" + 0.011*"fund" + 0.011*"called" + 0.009*"time"')]

In [17]:
ldamodel[corpus[1]]

[(0, 0.13155523), (1, 0.3362723), (2, 0.11121012), (4, 0.4198335)]

In [18]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)